## **Codebasics Resume Project Challenge 4** - Ad-hoc Analysis for AtliQ Hardware.
---

In [1]:
# Import important libraries
import numpy as np
import pandas as pd

In [2]:
# Read the files
customer = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\dim_customer.csv")
dim_product = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\dim_product.csv")
gross_price = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\fact_gross_price.csv")
sales_monthly = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\fact_sales_monthly.csv")
pre_invoice_deductions = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\fact_pre_invoice_deductions.csv")
manufacturing_cost = pd.read_csv(r"D:\CodeBasics\RPC_04\4_CSV_Files\fact_manufacturing_cost.csv")

## **Q1. Provide the list of markets in which customer "Atliq Exclusive" operates its business in the APAC region.**

In [3]:
markets = customer[(customer['customer'] == 'Atliq Exclusive') & (customer['region'] == 'APAC')]['market'].unique()

# Convert to DataFrame
df_Q1 = pd.DataFrame(markets, columns=['Market'])

# Display result
df_Q1

,Market
0,India
1,Indonesia
2,Japan
3,Philiphines
4,South Korea
5,Australia
6,Newzealand
7,Bangladesh


## **2. What is the percentage of unique product increase in 2021 vs. 2020?**

In [4]:
# Calculate unique products in 2020
sales_2020 = sales_monthly[sales_monthly['fiscal_year'] == 2020]
unique_products_2020 = sales_2020['product_code'].nunique()

# Calculate unique products in 2021
sales_2021 = sales_monthly[sales_monthly['fiscal_year'] == 2021]
unique_products_2021 = sales_2021['product_code'].nunique()

# Calculate the percentage increase
percentage_increase = ((unique_products_2021 - unique_products_2020) / unique_products_2020) * 100

# Create a DataFrame to display results
df_Q2 = pd.DataFrame({
    "UNIQUE_PRODUCTS_2020": [unique_products_2020],
    "UNIQUE_PRODUCTS_2021": [unique_products_2021],
    "CHANGE_PERCENTAGE": [f"{percentage_increase:.2f}%"]
})

df_Q2

,UNIQUE_PRODUCTS_2020,UNIQUE_PRODUCTS_2021,CHANGE_PERCENTAGE
0,245,334,36.33%


## **Q3. Provide a report with all the unique product counts for each segment and sort them in descending order of product counts.**

In [5]:
# Group by segment and count unique products
segment_product_counts = dim_product.groupby('segment')['product_code'].nunique().reset_index()

# Sort in descending order
segment_product_counts = segment_product_counts.sort_values(by='product_code', ascending=False)

# Rename columns for better readability
segment_product_counts.rename(columns={'product_code': 'product_count'}, inplace=True)

# Display the report
segment_product_counts

,segment,product_count
3,Notebook,129
0,Accessories,116
4,Peripherals,84
1,Desktop,32
5,Storage,27
2,Networking,9


## **Q4. Follow-up: Which segment had the most increase in unique products in 2021 vs 2020?**

In [6]:
# Merge sales data with product data
sales_product = pd.merge(sales_monthly, dim_product, on='product_code', how='left')

# Calculate unique products per segment in 2020
sales_2020 = sales_product[sales_product['fiscal_year'] == 2020]
segment_products_2020 = sales_2020.groupby('segment')['product_code'].nunique().reset_index()
segment_products_2020.rename(columns={'product_code': 'Product Count 2020'}, inplace=True)

# Calculate unique products per segment in 2021
sales_2021 = sales_product[sales_product['fiscal_year'] == 2021]
segment_products_2021 = sales_2021.groupby('segment')['product_code'].nunique().reset_index()
segment_products_2021.rename(columns={'product_code': 'Product Count 2021'}, inplace=True)

# Merge the two DataFrames
segment_products = pd.merge(segment_products_2020, segment_products_2021, on='segment', how='left')

# Calculate the difference
segment_products['Difference'] = segment_products['Product Count 2021'] - segment_products['Product Count 2020']

# Rename column for clarity
segment_products.rename(columns={'segment': 'Segment'}, inplace=True)

segment_products

,Segment,Product Count 2020,Product Count 2021,Difference
0,Accessories,69,103,34
1,Desktop,7,22,15
2,Networking,6,9,3
3,Notebook,92,108,16
4,Peripherals,59,75,16
5,Storage,12,17,5


## **Q5. Get the products that have the highest and lowest manufacturing costs.**

In [7]:
# Merge necessary columns from the manufacturing cost and product dimension tables
merged_data = pd.merge(manufacturing_cost, dim_product[['product_code', 'product']], on='product_code', how='left')

# Find the product with the highest manufacturing cost
highest_cost_product = merged_data.loc[merged_data['manufacturing_cost'].idxmax()]

# Find the product with the lowest manufacturing cost
lowest_cost_product = merged_data.loc[merged_data['manufacturing_cost'].idxmin()]

# Create a DataFrame to display the results
result_df = pd.DataFrame({
    'Product': [highest_cost_product['product'], lowest_cost_product['product']],
    'Manufacturing Cost': [highest_cost_product['manufacturing_cost'], lowest_cost_product['manufacturing_cost']],
    'Category': ['Highest', 'Lowest']
})

# Display the result
result_df

,Product,Manufacturing Cost,Category
0,AQ HOME Allin1 Gen 2,240.5364,Highest
1,AQ Master wired x1 Ms,0.8920,Lowest


## **Q6. Generate a report which contains the top 5 customers who received an average high pre_invoice_discount_pct for the fiscal year 2021 and in the Indian market.**

In [8]:
# Merge necessary tables
merged_data = pd.merge(pre_invoice_deductions, customer, on='customer_code', how='inner')
merged_data = merged_data[merged_data['fiscal_year'] == 2021]
merged_data = merged_data[merged_data['market'] == 'India']

# Calculate average discount percentage for each customer
average_discount = merged_data.groupby('customer')['pre_invoice_discount_pct'].mean().reset_index()

# Sort customers by average discount percentage in descending order
average_discount = average_discount.sort_values(by='pre_invoice_discount_pct', ascending=False)

# Get the top 5 customers
top_5_customers = average_discount.head(5)

# Display the report
top_5_customers

,customer,pre_invoice_discount_pct
10,Flipkart,0.3083
16,Viveks,0.3038
9,Ezone,0.3028
4,Croma,0.3025
1,Amazon,0.2933


## **Q7. Get the complete report of the Gross sales amount for the customer “Atliq Exclusive” for each month. This analysis helps to get an idea of low and high-performing months and take strategic decisions.**

In [9]:
# Merge necessary tables
merged_data = pd.merge(sales_monthly, customer, on='customer_code', how='inner')

# Filter data for "Atliq Exclusive" customer
atliq_exclusive_sales = merged_data[merged_data['customer'] == 'Atliq Exclusive']

# Merge the two dataframes
atliq_exclusive_sales = pd.merge(atliq_exclusive_sales, gross_price, on=['product_code', 'fiscal_year'], how='left')

# Calculate gross sales amount
atliq_exclusive_sales['gross_sales_amount'] = atliq_exclusive_sales['sold_quantity'] * atliq_exclusive_sales['gross_price']


# Convert the 'date' column to datetime objects if it's not already
atliq_exclusive_sales['date'] = pd.to_datetime(atliq_exclusive_sales['date'])

atliq_exclusive_sales

,date,product_code,customer_code,sold_quantity,fiscal_year,customer,platform,channel,market,sub_zone,region,gross_price,gross_sales_amount
0,2019-09-01,A0118150101,70002017,137,2020,Atliq Exclusive,Brick & Mortar,Direct,India,India,APAC,16.2323,2223.8251
1,2019-09-01,A0118150101,70003181,57,2020,Atliq Exclusive,Brick & Mortar,Direct,Indonesia,ROA,APAC,16.2323,925.2411
2,2019-09-01,A0118150101,70004069,9,2020,Atliq Exclusive,Brick & Mortar,Direct,Japan,ROA,APAC,16.2323,146.0907
3,2019-09-01,A0118150101,70006157,61,2020,Atliq Exclusive,Brick & Mortar,Direct,Philiphines,ROA,APAC,16.2323,990.1703
4,2019-09-01,A0118150101,70007198,51,2020,Atliq Exclusive,Brick & Mortar,Direct,South Korea,ROA,APAC,16.2323,827.8473
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80756,2021-08-01,A7321160303,70014142,72,2021,Atliq Exclusive,Brick & Mortar,Direct,Netherlands,NE,EU,42.8483,3085.0776
80757,2021-08-01,A7321160303,70015151,64,2021,Atliq Exclusive,Brick & Mortar,Direct,Norway,NE,EU,42.8483,2742.2912
80758,2021-08-01,A7321160303,70022084,260,2021,Atliq Exclusive,Brick & Mortar,Direct,USA,NaN,NaN,42.8483,11140.5580
80759,2021-08-01,A7321160303,70023031,166,2021,Atliq Exclusive,Brick & Mortar,Direct,Canada,NaN,NaN,42.8483,7112.8178


In [10]:
# Group by month and sum gross sales amount
monthly_sales = atliq_exclusive_sales.groupby(atliq_exclusive_sales['date'].dt.to_period('M'))['gross_sales_amount'].sum().reset_index()

# Rename columns for clarity
monthly_sales.rename(columns={'date': 'YYYY-MM', 'gross_sales_amount': 'Gross Sales Amount'}, inplace=True)

# Display the report
monthly_sales

,YYYY-MM,Gross Sales Amount
0,2019-09,4.496260e+06
1,2019-10,5.135902e+06
2,2019-11,7.522893e+06
3,2019-12,4.830405e+06
4,2020-01,4.740600e+06
5,2020-02,3.996228e+06
6,2020-03,3.787710e+05
7,2020-04,3.950354e+05
8,2020-05,7.838134e+05
9,2020-06,1.695217e+06


In [11]:
# Add a column for month name
monthly_sales['Month Name'] = monthly_sales['YYYY-MM'].dt.strftime('%b')

# Define fiscal year logic
# Fiscal year starts in September and ends in August
monthly_sales['Fiscal Year'] = monthly_sales['YYYY-MM'].apply(lambda x: f"FY{x.year + 1}" if x.month >= 9 else f"FY{x.year}")

# Sort by Month column to maintain sequence
monthly_sales = monthly_sales.sort_values(by="YYYY-MM").reset_index(drop=True)

# Format 'Gross Sales Amount' to two decimal places
monthly_sales['Gross Sales Amount'] = monthly_sales['Gross Sales Amount'].round(2)

# Reorder the columns to: Month, Month Name, Fiscal Year, Gross Sales Amount
monthly_sales = monthly_sales[['YYYY-MM', 'Month Name', 'Fiscal Year', 'Gross Sales Amount']]

# Display the updated DataFrame
print(monthly_sales)

    YYYY-MM Month Name Fiscal Year  Gross Sales Amount
0   2019-09        Sep      FY2020          4496259.67
1   2019-10        Oct      FY2020          5135902.35
2   2019-11        Nov      FY2020          7522892.56
3   2019-12        Dec      FY2020          4830404.73
4   2020-01        Jan      FY2020          4740600.16
5   2020-02        Feb      FY2020          3996227.77
6   2020-03        Mar      FY2020           378770.97
7   2020-04        Apr      FY2020           395035.35
8   2020-05        May      FY2020           783813.42
9   2020-06        Jun      FY2020          1695216.60
10  2020-07        Jul      FY2020          2551159.16
11  2020-08        Aug      FY2020          2786648.26
12  2020-09        Sep      FY2021         12353509.79
13  2020-10        Oct      FY2021         13218636.20
14  2020-11        Nov      FY2021         20464999.10
15  2020-12        Dec      FY2021         12944659.65
16  2021-01        Jan      FY2021         12399392.98
17  2021-0

## **Q8. In which quarter of 2020, got the maximum total_sold_quantity?**

In [12]:
# Filter data for the year 2020
sales_2020 = sales_monthly[sales_monthly['fiscal_year'] == 2020].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Convert 'date' column to datetime objects
sales_2020['date'] = pd.to_datetime(sales_2020['date'])

# Define fiscal quarters based on the fiscal year starting in September
def get_fiscal_quarter(date):
    month = date.month
    if month in [9, 10, 11]:  # Sep, Oct, Nov -> Q1
        return 1
    elif month in [12, 1, 2]:  # Dec, Jan, Feb -> Q2
        return 2
    elif month in [3, 4, 5]:  # Mar, Apr, May -> Q3
        return 3
    else:  # Jun, Jul, Aug -> Q4
        return 4

# Assign fiscal quarter
sales_2020['quarter'] = sales_2020['date'].apply(get_fiscal_quarter)

# Group by quarter and sum the sold quantity
quarterly_sales = sales_2020.groupby('quarter')['sold_quantity'].sum()

quarterly_sales

quarter
1    7005619
2    6649642
3    2075087
4    5042541
Name: sold_quantity, dtype: int64

In [13]:
# Find the quarter with the maximum total sold quantity
max_quarter = quarterly_sales.idxmax()
max_quantity = quarterly_sales.max()

# Display results
print(f"The quarter with the highest total sold quantity in fiscal year 2020 is Q{max_quarter} with {max_quantity} units.")

The quarter with the highest total sold quantity in fiscal year 2020 is Q1 with 7005619 units.


## **Q9. Which channel helped to bring more gross sales in the fiscal year 2021 and the percentage of contribution?**

In [14]:
# Merge necessary tables
merged_data = pd.merge(sales_monthly, gross_price, on=['product_code', 'fiscal_year'], how='left')

# Filter data for fiscal year 2021
sales_2021 = merged_data[merged_data['fiscal_year'] == 2021].copy()  # Using .copy() to avoid warning

# Calculate gross sales amount
sales_2021.loc[:, 'gross_sales_amount'] = sales_2021['sold_quantity'] * sales_2021['gross_price']

# Merge with customer data
sales_2021 = pd.merge(sales_2021, customer, on='customer_code', how='left')

# Group by channel and sum gross sales amount
channel_sales = sales_2021.groupby('channel', as_index=False)['gross_sales_amount'].sum()

# Convert gross sales amount to millions
channel_sales['gross_sales_amount_million'] = channel_sales['gross_sales_amount'] / 1e6

# Calculate the total gross sales for 2021
total_gross_sales_2021 = channel_sales['gross_sales_amount_million'].sum()

# Calculate the percentage contribution of each channel
channel_sales['percentage_contribution'] = (channel_sales['gross_sales_amount_million'] / total_gross_sales_2021) * 100

# Sort by percentage contribution in descending order
channel_sales = channel_sales.sort_values(by='percentage_contribution', ascending=False)

# Format gross sales amount and percentage contribution to 2 decimal places
channel_sales['gross_sales_amount_million'] = channel_sales['gross_sales_amount_million'].map(lambda x: f"{x:.2f}M")
channel_sales['percentage_contribution'] = channel_sales['percentage_contribution'].map(lambda x: f"{x:.2f}%")

# Drop the original gross sales amount column
channel_sales.drop(columns=['gross_sales_amount'], inplace=True, errors='ignore')

# Reset index after sorting
channel_sales.reset_index(drop=True, inplace=True)

# Display the final result
channel_sales

,channel,gross_sales_amount_million,percentage_contribution
0,Retailer,1219.08M,73.23%
1,Direct,257.53M,15.47%
2,Distributor,188.03M,11.30%


## **Q10. Get the Top 3 products in each division that have a high total_sold_quantity in the fiscal_year 2021?**

In [15]:
# Merge necessary tables
merged_data = pd.merge(sales_monthly, dim_product, on='product_code', how='left')

# Filter data for fiscal year 2021
sales_2021 = merged_data[merged_data['fiscal_year'] == 2021].copy()  # Use .copy() to avoid warning

# Group by division and product, sum sold quantity
division_products = sales_2021.groupby(['division', 'product_code', 'product'])['sold_quantity'].sum().reset_index()

# Sort by division and sold quantity in descending order
division_products = division_products.sort_values(['division', 'sold_quantity'], ascending=[True, False])

# Get the top 3 products for each division
top_3_products = division_products.groupby('division').head(3).copy()  # Use .copy() to avoid warning

# Add a rank column within each division based on total_sold_quantity (without decimal)
top_3_products.loc[:, 'rank'] = top_3_products.groupby('division')['sold_quantity'].rank(method='dense', ascending=False).astype(int)

# Display the result
top_3_products

,division,product_code,product,sold_quantity,rank
13,N & S,A6720160103,AQ Pen Drive 2 IN 1,701373,1
15,N & S,A6818160202,AQ Pen Drive DRC,688003,2
16,N & S,A6819160203,AQ Pen Drive DRC,676245,3
114,P & A,A2319150302,AQ Gamers Ms,428498,1
125,P & A,A2520150501,AQ Maxima Ms,419865,2
128,P & A,A2520150504,AQ Maxima Ms,419471,3
208,PC,A4218110202,AQ Digit,17434,1
220,PC,A4319110306,AQ Velocity,17280,2
214,PC,A4218110208,AQ Digit,17275,3


---

# Thank You

### I appreciate your time and attention! Feel free to connect for any questions or discussions.

#### Connect with me:
- [GitHub](https://github.com/ANIRUDDHA-BISWAS)
- [LinkedIn](https://www.linkedin.com/in/aniruddha-biswas/)
